## Day 18 Lavaduct Lagoon

### Part 1

1. Shoelace Formula:
- Determines the area $A$ of a simple polygon using the coordinates of its vertices
$$ A = \frac{1}{2}\sum_{i=1}^{n} y_i(x_{i-1} - x_{i+1}) $$

2. Pick's Theorem:
- Uses the number of integer points $i$ inside a polygon, and on its edges $b$, to find its area $A$.
$$ A = i + \frac{b}{2} - 1 $$
This can be arranged to find:
$i$ = $A - \frac{b}{2} + 1$

The total area to output will be $2\times (A-i+1) + i$.
We simulate the instructions in the input and mark the points encountered in each instruction path as visited. In the end, we find the number of distinct points visited, which will be our $b$ -- the number of integer points of the edges (for Pick's Theorem), as well as the coordinates of the vertices (the point where we are at after each instruction -- for Shoelace Formula).
We use the Shoelace Formula to find the area $A$, and from there we can find the total area to output.

Our algorithm for finding the number of integer points on the edge of the polygon can be optimised once we realise that there is no chance of double-counting the points, since they don't overlap -- the second iteration of the part 1 code.

In [109]:
# Part 1

def shoelace_formula(corners):
    area = 0.0
    x, y = list(zip(*corners))
    
    for i in range(len(corners)):
        area += y[i]*(x[(i-1)%len(corners)] - x[(i+1)%len(corners)])

    return abs(area) / 2.0

plan = open('day_18_input.txt').read().splitlines()

DIRECTIONS = {'L':(0,-1), 'R':(0,1), 'U':(-1,0), 'D':(1,0)} #dr, dc
curr = (0,0) #r, c
visited = set() #r, c, dr, dc
vertices = [curr]

for instruction in plan:
    
    direction, step, colour = instruction.split()
    step = int(step)

    dr, dc = DIRECTIONS[direction]
    
    if dr != 0 and dc == 0: #changing the row
        if dr > 0:
            for i in range(curr[0]+1, curr[0]+(dr*step)+1):
                visited.add((i, curr[1], dr, dc))
            curr = (i, curr[1])
        else:
            for i in range(curr[0]+(dr*step), curr[0]):
                visited.add((i, curr[1], dr, dc))
            curr = (curr[0]+(dr*step), curr[1])
    elif dr == 0 and dc != 0: #changing column
        if dc > 0:
            for i in range(curr[1]+1, curr[1]+(dc*step)+1):
                visited.add((curr[0], i, dr, dc))
            curr = (curr[0], i)
        else:
            for i in range(curr[1]+(dc*step), curr[1]):
                visited.add((curr[0], i, dr, dc))
            curr = (curr[0], curr[1]+(dc*step))
    vertices.append(curr)

edge_points = len({(r,c) for (r,c,_,_) in visited})
area = shoelace_formula(vertices)

internal_integer_points = area - edge_points/2 + 1

print( 2*(area - internal_integer_points + 1) + internal_integer_points )

#50465

50465.0


In [100]:
# Part 1 - Optimisation

def shoelace_formula(corners):
    area = 0.0
    x, y = list(zip(*corners))
    
    for i in range(len(corners)):
        area += y[i]*(x[(i-1)%len(corners)] - x[(i+1)%len(corners)])

    return abs(area) / 2.0

from collections import defaultdict

plan = open('day_18_input.txt').read().splitlines()

DIRECTIONS = {'L':(0,-1), 'R':(0,1), 'U':(-1,0), 'D':(1,0)} #dr, dc
curr = (0,0) #r, c
vertices = [curr]
edge_points = 0

for instruction in plan:
    
    direction, step, colour = instruction.split()
    step = int(step)

    dr, dc = DIRECTIONS[direction]
    if dr != 0 and dc == 0: #changing the row
        if dr > 0:
            edge_points += abs(dr*step)
            curr = (curr[0]+(dr*step), curr[1])
        else:
            edge_points += abs(dr*step)
            curr = (curr[0]+(dr*step), curr[1])
    elif dr == 0 and dc != 0: #changing column
        if dc > 0:
            edge_points += abs(dc*step)
            curr = (curr[0], curr[1]+(dc*step))
        else:
            edge_points += abs(dc*step)
            curr = (curr[0], curr[1]+(dc*step))
    vertices.append(curr)

print(edge_points)
area = shoelace_formula(vertices)
print(vertices)
internal_integer_points = area - edge_points/2 + 1

print( 2*(area - internal_integer_points + 1) + internal_integer_points )

#50465

3314
[(0, 0), (0, 6), (-3, 6), (-3, 3), (-6, 3), (-6, -3), (-11, -3), (-11, -12), (-15, -12), (-15, -7), (-18, -7), (-18, -4), (-28, -4), (-28, 0), (-26, 0), (-26, 2), (-15, 2), (-15, 6), (-20, 6), (-20, 11), (-11, 11), (-11, 15), (-7, 15), (-7, 17), (0, 17), (0, 20), (-4, 20), (-4, 27), (-7, 27), (-7, 32), (-14, 32), (-14, 35), (-10, 35), (-10, 42), (-4, 42), (-4, 48), (-12, 48), (-12, 50), (-17, 50), (-17, 54), (-21, 54), (-21, 50), (-29, 50), (-29, 44), (-24, 44), (-24, 40), (-29, 40), (-29, 35), (-24, 35), (-24, 31), (-29, 31), (-29, 26), (-25, 26), (-25, 23), (-21, 23), (-21, 20), (-25, 20), (-25, 17), (-27, 17), (-27, 7), (-32, 7), (-32, 11), (-39, 11), (-39, 13), (-42, 13), (-42, 17), (-32, 17), (-32, 20), (-36, 20), (-36, 28), (-41, 28), (-41, 31), (-46, 31), (-46, 35), (-52, 35), (-52, 42), (-55, 42), (-55, 47), (-65, 47), (-65, 51), (-55, 51), (-55, 55), (-57, 55), (-57, 59), (-63, 59), (-63, 67), (-66, 67), (-66, 69), (-72, 69), (-72, 77), (-77, 77), (-77, 80), (-80, 80), (-

### Part 2

All we really need to do is to convert the 'colour code' in hexadecimal into decimal, and handle the last hexadecimal digit by mapping it to the corresponding direction. From there, we can reuse our part 1 code and get the answer.

In [108]:
# Part 2

def shoelace_formula(corners):
    area = 0.0
    x, y = list(zip(*corners))
    
    for i in range(len(corners)):
        area += y[i]*(x[(i-1)%len(corners)] - x[(i+1)%len(corners)])

    return abs(area) / 2.0

plan = open('day_18_input.txt').read().splitlines()

DIRECTIONS = {'L':(0,-1), 'R':(0,1), 'U':(-1,0), 'D':(1,0)} #dr, dc
DIRECTION_CONVERSION = {'0':'R', '1':'D','2':'L','3':'U'}
curr = (0,0) #r, c
vertices = [curr]
edge_points = 0

for instruction in plan:
    
    fake_dir, fake_step, hex_code = instruction.split()
    
    hex_code = hex_code.strip('()#')

    direction = DIRECTION_CONVERSION[hex_code[-1]]
    step = int(hex_code[:-1], base=16)

    dr, dc = DIRECTIONS[direction]
    
    if dr != 0 and dc == 0: #changing the row
        if dr > 0:
            edge_points += abs(dr*step)
            curr = (curr[0]+(dr*step), curr[1])
        else:
            edge_points += abs(dr*step)
            curr = (curr[0]+(dr*step), curr[1])
    elif dr == 0 and dc != 0: #changing column
        if dc > 0:
            edge_points += abs(dc*step)
            curr = (curr[0], curr[1]+(dc*step))
        else:
            edge_points += abs(dc*step)
            curr = (curr[0], curr[1]+(dc*step))
    vertices.append(curr)

area = shoelace_formula(vertices)

internal_integer_points = area - edge_points/2 + 1

print( 2*(area - internal_integer_points + 1) + internal_integer_points )

#82712746433310

82712746433310.0
